In [ ]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup as bs
from scipy import stats
import datetime

In [ ]:
tick = pd.read_csv("all_stocks")

In [ ]:
final = pd.DataFrame(columns=['Ticker','Price','Trailing PE ratio','Forward PE ratio','Price To Book',\
                                'Price To Sale','EV/EBITDA','EV/GP','Working Capital','Shares to buy'])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = tick["0"][i]
        stock_info = yf.Ticker(ticker)
        d = yf.Ticker(tick["0"][i]).history(period = "1mo")
        # Get the income statement for the stock

        info = stock_info.info
        fpe = info["forwardPE"]
        tpe = info["trailingPE"]
        fin = stock_info.info
        liab = stock_info.balance_sheet
        price_to_book = fin['priceToBook']
        price_to_sale = fin['priceToSalesTrailing12Months']
        ev_ebidta = fin['enterpriseToEbitda']
        enterprise_value = fin['enterpriseValue']
        gp = fin['grossProfits']
        ev_gp = enterprise_value/gp
        current_assets = liab[liab.index=='Current Assets'].iloc[0,0]
        current_liab = liab[liab.index=='Current Liabilities'].iloc[0,0]
        working_capital = current_assets/current_liab 

            #Ratio endettement (liabilities/assets)
            #Working capital (currentAssets/currentLiability) Voir si depense sont capapble d'etre couverte dans la prochaine annee
            #In balance_sheet we have: 
                #Current assets
                #Current liabilities

    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    
    d.reset_index(inplace = True)
    try: 
        d['Ticker'] = tick['0'][i]
        d["Price"] = d['Close']
        d['Trailing PE ratio'] = tpe
        d['Forward PE ratio'] = fpe
        d['Price To Book'] = price_to_book
        d['Price To Sale'] = price_to_sale
        d['EV/EBITDA'] = ev_ebidta
        d['EV/GP'] = ev_gp
        d['Working Capital'] = working_capital
        d = d.iloc[-1]
    except IndexError: 
        continue
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Price To Book', 'Price To Sale','EV/EBITDA','EV/GP','Working Capital','Shares to buy']), 
        ignore_index = True
    ) 

In [ ]:
final.to_csv("value_invest_advanced_sheet")